## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using Plots
using IJulia

In [2]:
using TensorFlow
using Distributions
using StatsBase
using StaticArrays

In [3]:
using DensityEstimationML
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

only (generic function with 1 method)

In [4]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(dataset))
sess = est.sess
run(sess, est.pdf, Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

dytn = <Tensor yt:1 shape=unknown dtype=Float32>
t_col = 

2017-09-20 20:44:07.097554: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:44:07.097584: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:44:07.097590: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:44:07.300260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-20 20:44:07.300763: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

<Tensor t_1:1 shape=(?) dtype=Float32>


LoadError: [91mKeyError: key Any not found[39m

In [8]:
gr=get_def_graph()

Graph(Ptr{Void} @0x0000000007300000)

In [24]:
gradients(gr["yt"], gr["t"])[1,:] |> get_shape

TensorShape[?]

In [26]:
[1 2 3; 4 54 6][1,:ii

3-element Array{Int64,1}:
 1
 2
 3

In [12]:
println.(sort(gr |> keys |> collect))

Add
Add_2
Add_3
Cast
Cast_2
Cast_3
Cast_4
Cast_5
Cast_6
Const_4
Const_5
Exp
Exp_2
Exp_3
MatMul
MatMul_2
MatMul_3
MatMul_4
MatMul_5
MatMul_6
Sigmoid
Sigmoid_2
Sigmoid_3
Sub
W_2
W_2/Assign
W_2/Assign/Const
W_2_squared
W_3
W_3/Assign
W_3/Assign/Const_3
W_3_squared
b_2
b_2/Assign
b_2/Assign/Const_2
denominator
denominator/range
denominator/rank
gradients/Const
gradients/Fill
gradients/Shape
smax
smin
t
t_1
t_1/Const_6
t_1/Const_7
t_1/Sub_2
ysmax
ysmin
yt


51-element Array{Void,1}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 ⋮      
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing

In [ ]:
"""
Function returning a function that will display a running plot.
WARNING: Introducting or removing any variables is not supported.
And will silently error.
"""
function running_plot()
    epochs = Int[]
    record = Dict()
    function inner(epoch, vars::Associative)
        for (var, values) in vars
            value = only(values) #Incase it was an array
            past = get!(record, var) do
                typeof(value)[]
            end
            push!(past, value)
        end
        push!(epochs, epoch)
        
        IJulia.clear_output(true)
        plot(epochs, hcat(values(record)...); label=hcat(keys(vars)...), layout=length(vars)) |> IJulia.display       
    end
end


In [ ]:
function demonstration_plot(est, dataset, data, args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    @show typeof(data)
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

In [ ]:
function demo(dataset, layers, epochs=20_000; max_conditioning_epochs=2000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))

    condition!(est; max_epochs = max_conditioning_epochs)
    println("Conditioning Done")
    Plots.gr()
    fit!(est, data; epochs=epochs, callback=running_plot())
    println("Fitting Done")
    plotly()
    demonstration_plot(est, dataset, data) |> IJulia.display
    
    est    
end

In [ ]:
est=demo(GenerateDatasets.Likas1(), [64, 64, 256], 1_000)

In [ ]:
let 
    sess = Session(Graph())
    X=constant([1.0,2,3])
    y=Ops.mul(X,X)
    @show y
end

In [ ]:
loglikelihood(GenerateDatasets.Likas1(), [2.0])

In [ ]:
GenerateDatasets.Likas1()

In [ ]:
sort(component_weights(est))

In [ ]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

In [ ]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 10_000)

In [ ]:
demo(Arcsine(1,4), [64,64], 20_000)

In [ ]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(GenerateDatasets.Likas3()))

LoadError: [91mUndefVarError: GenerateDatasets not defined[39m

In [5]:
@show loglikelihood(dataset, data)

loglikelihood(dataset, data) = 16094.379124341185


16094.379124341185

In [6]:
est = NeuralDensityEstimator([64], approximate_support(GenerateDatasets.Likas3()))

2017-09-20 20:26:25.280835: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:26:25.280863: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:26:25.280869: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-20 20:26:25.485456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-20 20:26:25.485961: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

LoadError: [91mOn worker 2:
[91mPython error: 'NoneType' object has no attribute 'dtype'[39m
py_gradients at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/TensorFlow/src/py.jl:45
#9 at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/TensorFlow/src/TensorFlow.jl:171
#110 at ./distributed/process_messages.jl:275 [inlined]
run_work_thunk at ./distributed/process_messages.jl:56
run_work_thunk at ./distributed/process_messages.jl:65 [inlined]
#96 at ./event.jl:73[39m

In [ ]:
condition!(est)

In [ ]:

fit!(est, data'; epochs=10_000, callback=running_plot())

In [ ]:

fit!(est, data; epochs=10_000, callback=running_plot())
println("Fitting Done")

In [7]:
sess = est.sess
run(sess, est.pdf, Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

LoadError: [91mUndefVarError: est not defined[39m

In [ ]:
sess = est.sess
run(sess, sess.graph["numerator"], Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

In [ ]:
data

In [ ]:
run(sess, gather(constant(data), 1))

In [ ]:
plotly()

In [ ]:
"""
    meshgrid(xs, ys, zfun)

Evalates `zfun(x,y)` at each point in `xs`, and `ys`.
Returns 3 vectors, a list of x points a list of y points and the value of z at that point.
"""
function meshgrid(xs, ys, zfun) 
    # There is a cute generalisation of this with a `@generated` function
    xpoints = eltype(xs)[]
    ypoints = eltype(ys)[]
    zpoints = typeof(zfun(xs[1],ys[1]))[]
    sizehint!.([xpoints, ypoints, zpoints], length(xs)*length(ys))
    for x in xs, y in ys
        push!(xpoints,x)
        push!(ypoints,y)
        push!(zpoints,zfun(x,y))
    end
    
    xpoints, ypoints, zpoints
end


In [ ]:
X=-0.1:0.01:0.3
Y=-0.1:0.01:0.3
scatter3d(meshgrid(X,Y, (x,y)->pdf(est, [x,y]))...)

In [ ]:
pdf(est, SMatrix{1,2}([0.1 0.1]))

In [ ]:
est

In [ ]:
size([0.1 0.1])

In [ ]:
?SArray